## Prime Generation

In [ ]:
from nprime.pyprime import miller_rabin
from math import gcd as bltin_gcd
from secrets import randbits, randbelow
from sympy import mod_inverse

In [ ]:
def generate_prime_candidate(len):
    p_can = randbits(len)
    
    p_can |= (1 << len - 1) | 1
    return p_can

In [ ]:
def gen_prime_number(len=1024):
    p = 4
    while not miller_rabin(p, t=128):
        p = generate_prime_candidate(len)
    return p

# Coprime generation functions
def coprime2(a, b):
    return bltin_gcd(a, b) == 1

def gen_coprime(n):
    m = randbelow(n - 1) + 2
    while not coprime2(m, n):
        m = randbelow(n - 1) + 2
    return m

In [ ]:
p = gen_prime_number()
q = gen_prime_number()

In [ ]:
print(f"p: { p }\nq: { q }\np * q: { p * q }")

## Encode message as a number

In [ ]:
def str_to_int(in_string):
    int_list = [None] * len(in_string)
    for index, c in enumerate(in_string.upper()):
        int_list[index] = ord(c)
    l_to_i = [str(num) for num in int_list]
    l_to_i = int("".join(l_to_i))
    return l_to_i

In [ ]:
test = str_to_int("Hello")
print(test)

## Decode the number as a string

In [ ]:
def int_to_str(in_int):
    i_str = str(in_int)
    new_str = ""
    for x in range(len(i_str) // 2):
        new_str += (chr(int(i_str[x*2:x*2 + 2])))
    return new_str.lower()

In [ ]:
test = "hello world!"
test_e = str_to_int(test)
test_d = int_to_str(test_e)

print(f"'{ test }' is encoded as '{ test_e }', and decoded as '{ test_d }'")

# Testing our RSA

In [ ]:
# Generating our initial primes
p = gen_prime_number(1024)
q = gen_prime_number(1024)
print(p)
print(q)

In [ ]:
# Calculate n
n = p * q
print(n)

In [ ]:
# Calculate totient of n tot = (p - 1) * (q - 1)
tot = (p - 1) * (q - 1)
print(tot)

In [ ]:
# Generate coprime for n
e  = gen_coprime(n)
print(e)

In [ ]:
#Find the value d for private key pair (decryption key)
# Hardest one to calculate by brute force
d = mod_inverse(e, tot)
print(d)

In [ ]:
# Create our key pairs
pub_k = (n, e)
pri_k = (n, d)

In [ ]:
# Now let's create a string and encode it as an integer
input_string = "Quick brown fox jumped over the lazy"
inted_str = str_to_int(input_string)
print(inted_str)

In [ ]:
# Encrypt the message value by using the public key
encoded_string = pow(inted_str, pub_k[1], pub_k[0])
print(encoded_string)

In [ ]:
# Decrypt the value to the message using private key
decrypted_string = int_to_str(pow(encoded_string, pri_k[1], pri_k[0]))
print(decrypted_string)

In [ ]:
# Let's do both steps in once
enc = pow(inted_str, pri_k[1], pri_k[0])
dec = int_to_str(pow(enc, pub_k[1], pub_k[0]))

In [ ]:
# Output
print(f"{ input_string } was represented as { inted_str }\n")
print(f"Then it was encoded as : { enc }\n")
print(f"Finally it was decoded as : { dec }")

## Testing with smaller key size

In [ ]:
p = gen_prime_number(128)
q = gen_prime_number(128)

print(p)
print(q)

In [ ]:
n = p * q
print(n)

In [ ]:
tot = (p - 1) * (q - 1)
print(tot)

In [ ]:
e = gen_coprime(n)
print(e)

In [ ]:
d = mod_inverse(e, tot)
print(d)

In [ ]:
pub_k = (n, e)
pri_k = (n, d)

In [ ]:
input_string = str_to_int("cheeki breeki iv damke!")
enc = pow(input_string, pub_k[1], pub_k[0])
print(enc)

In [ ]:
dec = pow(enc, pri_k[1], pri_k[0])
print(dec)

In [ ]:
output = int_to_str(dec)
print(output)

## Better Functions

In [12]:
from nprime.pyprime import miller_rabin
from math import gcd as bltin_gcd
from secrets import randbits, randbelow
from sympy import mod_inverse

def generate_prime_candidate(len):
    p_can = randbits(len)
    
    p_can |= (1 << len - 1) | 1
    return p_can

def gen_prime_number(len=1024):
    p = 4
    while not miller_rabin(p, t=128):
        p = generate_prime_candidate(len)
    return p

# Coprime generation functions
def coprime2(a, b):
    return bltin_gcd(a, b) == 1

def gen_coprime(n):
    m = randbelow(n - 1) + 2
    while not coprime2(m, n):
        m = randbelow(n - 1) + 2
    return m

def str_to_int(in_string):
    int_list = [None] * len(in_string)
    for index, c in enumerate(in_string.upper()):
        int_list[index] = ord(c)
    l_to_i = [str(num) for num in int_list]
    l_to_i = int("".join(l_to_i))
    return l_to_i

def int_to_str(in_int):
    i_str = str(in_int)
    new_str = ""
    for x in range(len(i_str) // 2):
        new_str += (chr(int(i_str[x*2:x*2 + 2])))
    return new_str.lower()

In [13]:
def generate_key_pair(len=128):
    p = gen_prime_number(len)
    q = gen_prime_number(len)
    
    n = p * q
    tot = (p - 1) * (q - 1)
    
    # Generate coprimes until a modular inversible d is found
    while True:
        try:
            e = gen_coprime(n)
            d = mod_inverse(e, tot)
        except ValueError:
            continue
        break
    
    public = (e, n)
    private = (d, n)
    
    key_pair = (public, private)
    return key_pair

def encrypt_RSA(message, public_key):
    inted_msg = str_to_int(message)
    return pow(inted_msg, public_key[0], public_key[1])

def decrypt_RSA(encrypted_msg, private_key):
    return int_to_str(pow(encrypted_msg, private_key[0], private_key[1]))

In [14]:
pub, pri = generate_key_pair(128)
print(pub)
print(pri)

(41682329846483988954343720222763547220138872664711517059939688366412709145863, 46691346968120214251603800058165092998096201439385296922114843889684667967321)
(1073904220577678722453188317142572033897427760937520899921162625235379158327, 46691346968120214251603800058165092998096201439385296922114843889684667967321)


In [15]:
enc = encrypt_RSA("Hello world", pub)
print(enc)

4042230546172913177453089899408352414129872939637626205902186540232290789851


In [16]:
dec = decrypt_RSA(enc, pri)
print(dec)

hello world
